In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cartopy
from tqdm import tqdm
from sklearn.neighbors import KernelDensity

## 1. Select the Ginea-Pig (Ensemble Member #23)

In [4]:
pset_23 = xr.open_zarr('/storage/shared/oceanparcels/output_data/data_Claudio/NEMO_Ensemble/2011/PGS_2011_023.zarr')

In [5]:
# Extract the coordinates of the particles at diffferent depths
level_indexes = {}
dz = 100
n_levels = 16

for i in range(n_levels):
    level_indexes[i*dz] = pset_23.trajectory[i::n_levels]

In [6]:
p23 = pset_23.isel(trajectory=level_indexes[100])

## 2. Function to get indexes of neighbour particles

In [7]:
def neighbour_particles(pset, lon, lat, delta_deg):
    """
    Function to find the particles in a certain radius around a point
    pset: xarray dataset with the particle data
    lon: longitude of the point
    lat: latitude of the point
    delta_deg: radius around the point in degrees
    
    Returns: the indexes of the particles in the radius around the point
    """
    
    lon_conditional = np.logical_and(pset.lon[:,0]>=lon-delta_deg, pset.lon[:,0]<=lon + delta_deg)
    lat_conditional = np.logical_and(pset.lat[:,0]>=lat-delta_deg, pset.lat[:,0]<=lat + delta_deg)
    s = np.where(np.logical_and(lon_conditional, lat_conditional))[0]
    return s

## 3. Build Dataset $\forall$ members